# Testing Python Bindings

In [1]:
import torch
import pandas
import tensortracker
import torchvision

rand_seed = 1234

In [12]:
mnist_data = torchvision.datasets.MNIST(root="./temp/", download=True, transform=torchvision.transforms.ToTensor())

In [15]:
data_loader = torch.utils.data.DataLoader(mnist_data, batch_size=4)

x, y = next(iter(data_loader))

In [2]:
import torch.nn as nn
import torch.nn.functional as F
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 16 * 16, 256),
            nn.ReLU(),
            nn.Linear(256, 10)
            
        )

    def forward(self, x):
        x = self.conv(x)
        x = self.fc(x)
        return x
    # def __init__(self):
    #     super(Net, self).__init__()
    #     self.conv1 = nn.Conv2d(3, 32, 3, 1)
    #     self.conv2 = nn.Conv2d(32, 64, 3, 1)
    #     self.dropout1 = nn.Dropout(0.25)
    #     self.dropout2 = nn.Dropout(0.5)
    #     self.fc1 = nn.Linear(64 * 16 * 16, 256)
    #     self.fc2 = nn.Linear(256, 10)

    # def forward(self, x):
    #     x = self.conv1(x)
    #     x = F.relu(x)
    #     x = self.conv2(x)
    #     x = F.relu(x)
    #     x = F.max_pool2d(x, 2)
    #     # x = self.dropout1(x)
    #     x = torch.flatten(x, 1)
    #     x = self.fc1(x)
    #     x = F.relu(x)
    #     # x = self.dropout2(x)
    #     x = self.fc2(x)
    #     output = F.log_softmax(x, dim=1)
    #     return x

In [3]:
import safetensors.torch as st

def train(model, device, train_loader, optimizer, epoch):
    criterion = torch.nn.CrossEntropyLoss()
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10000 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [4]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [5]:
use_accel = torch.accelerator.is_available()

torch.manual_seed(rand_seed)

if use_accel:
    print("Using accel")
    device = torch.accelerator.current_accelerator()
else:
    print("Using CPU")
    device = torch.device("cpu")

Using accel


In [6]:
train_config = {'batch_size': 64}
test_config = {'batch_size': 1000}

if use_accel:
    accel_kwargs = {'num_workers': 1,
                    'persistent_workers': True,
                    'pin_memory': True,
                    'shuffle': True}
    train_config.update(accel_kwargs)
    test_config.update(accel_kwargs)

# transform=torchvision.transforms.Compose([
#     torchvision.transforms.ToTensor(),
#     torchvision.transforms.Normalize((0.1307,), (0.3081,))
#     ])

transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # RGB mean/std
    ])

train_data = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_data = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# train_data = torchvision.datasets.MNIST('./temp/', train=True, download=True, transform=transform)
# test_data = torchvision.datasets.MNIST('./temp/', train=False, download=True, transform=transform)


train_loader = torch.utils.data.DataLoader(train_data, **train_config)
test_loader = torch.utils.data.DataLoader(test_data, **test_config)

In [7]:
model = Net().to(device)
# optimizer = torch.optim.Adadelta(model.parameters(), lr=1.0)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

In [9]:
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)
for epoch in range(1, 50):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    st.save_model(model, f"./results/cifar/checkpoints/model_{epoch}_.safetensors")


Train Epoch: 1 [0/50000 (0%)]	Loss: 1.211076

Test set: Average loss: -4.1470, Accuracy: 6562/10000 (66%)

Train Epoch: 2 [0/50000 (0%)]	Loss: 0.888293

Test set: Average loss: -4.9134, Accuracy: 6778/10000 (68%)

Train Epoch: 3 [0/50000 (0%)]	Loss: 0.515155

Test set: Average loss: -6.2396, Accuracy: 7000/10000 (70%)

Train Epoch: 4 [0/50000 (0%)]	Loss: 0.396702

Test set: Average loss: -7.9904, Accuracy: 6962/10000 (70%)

Train Epoch: 5 [0/50000 (0%)]	Loss: 0.291962

Test set: Average loss: -10.4366, Accuracy: 6995/10000 (70%)

Train Epoch: 6 [0/50000 (0%)]	Loss: 0.102944

Test set: Average loss: -12.3730, Accuracy: 7001/10000 (70%)

Train Epoch: 7 [0/50000 (0%)]	Loss: 0.137352

Test set: Average loss: -13.6900, Accuracy: 6854/10000 (69%)

Train Epoch: 8 [0/50000 (0%)]	Loss: 0.089246

Test set: Average loss: -15.4412, Accuracy: 6984/10000 (70%)

Train Epoch: 9 [0/50000 (0%)]	Loss: 0.026172

Test set: Average loss: -17.2976, Accuracy: 7045/10000 (70%)

Train Epoch: 10 [0/50000 (0%)]	L

KeyboardInterrupt: 

In [53]:
import safetensors.torch as st

st.save_model(model, "temp/checkpoints/model_1_0_.safetensors")